In [1]:
import matplotlib.pyplot as plt
from sklearn import metrics 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

import re
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Print files in current working directory 
os.chdir("/data/team15/data")
ebb_1 = pd.read_csv("ebb_set1.csv")
ebb_2 = pd.read_csv("ebb_set2.csv")
eval_set = pd.read_csv("eval_set.csv")
os.listdir(os.curdir)

['redemptions_eval_set.csv',
 'activations_eval_set.csv',
 'auto_refill_eval_set.csv',
 'lease_history_ebb_set1.csv',
 'ivr_calls_eval_set.csv',
 'ebb_set2.csv',
 'support_eval_set.csv',
 'deactivations_ebb_set2.csv',
 'reactivations_ebb_set1.csv',
 'redemptions_ebb_set1.csv',
 'phone_data_ebb_set1.csv',
 'catboost_info',
 'loyalty_program_ebb_set2.csv',
 'lease_history_ebb_set2.csv',
 'interactions_ebb_set1.csv',
 'redemptions_ebb_set2.csv',
 'suspensions_ebb_set1.csv',
 'gh_dataset_eval.csv',
 'phone_data_eval_set.csv',
 'reactivations_eval_set.csv',
 'network_eval_set.csv',
 'notifying_eval_set.csv',
 'lease_history_eval_set.csv',
 'network_ebb_set2.csv',
 'suspensions_eval_set.csv',
 'deactivations_eval_set.csv',
 'auto_refill_ebb_set1.csv',
 'phone_data_ebb_set2.csv',
 'interactions_ebb_set2.csv',
 'loyalty_program_eval_set.csv',
 'activations_ebb_set2.csv',
 'auto_refill_ebb_set2.csv',
 'ivr_calls_ebb_set2.csv',
 'throttling_ebb_set2.csv',
 'suspensions_ebb_set2.csv',
 'eval_set.

In [3]:
ebb_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49168 entries, 0 to 49167
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer           48764 non-null  object 
 8   operating_system       48764 non-null  object 
 9   language_preference    4741 non-null   object 
 10  opt_out_email          2856 non-null   float64
 11  opt_out_loyalty_email  79 non-null     float64
 12  opt_out_loyalty_sms    84 non-null     float64
 13  opt_out_mobiles_ads    24545 non-null  float64
 14  opt_out_phone          4311 non-null   float64
 15  st

In [4]:
ebb_1_act = pd.read_csv("activations_ebb_set1.csv")
ebb_2_act = pd.read_csv("activations_ebb_set2.csv")
eval_act = pd.read_csv("activations_eval_set.csv")
#ebb_1_act.info()
#ebb_1_act.head()

def count_act_times(df):
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='activation_counts')
    df = counts.drop_duplicates(subset=['customer_id'])
    return df

ebb_1_act = count_act_times(ebb_1_act)
ebb_2_act = count_act_times(ebb_2_act)
eval_act = count_act_times(eval_act)

#ebb_1_act.head()

ebb_1_o_act = ebb_1.merge(ebb_1_act, how='left', on='customer_id')
ebb_2_o_act = ebb_2.merge(ebb_2_act, how='left', on='customer_id')
eval_o_act = eval_set.merge(eval_act, how='left', on='customer_id')
ebb_1_o_act.head()

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,opt_out_loyalty_email,opt_out_loyalty_sms,opt_out_mobiles_ads,opt_out_phone,state,total_revenues_bucket,marketing_comms_1,marketing_comms_2,ebb_eligible,activation_counts
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,2.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,1.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,NaN,NaN,-1.0,NaN,TN,3,NaN,NaN,1.0,5.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,NaN,NaN,1.0,NaN,AL,1,NaN,NaN,1.0,1.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,NaN,NaN,1.0,NaN,MS,2,NaN,NaN,1.0,1.0


In [5]:
ebb_1_deact = pd.read_csv("deactivations_ebb_set1.csv")
ebb_2_deact = pd.read_csv("deactivations_ebb_set2.csv")
eval_deact = pd.read_csv("deactivations_eval_set.csv")
#ebb_1_deact.head()
#ebb_1_deact.info()
def count_times(df):
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='deactivation_counts')
    df = counts.drop_duplicates(subset=['customer_id'])
    return df

ebb_1_deact = count_times(ebb_1_deact)
ebb_2_deact = count_times(ebb_2_deact)
eval_deact = count_times(eval_deact)

#ebb_1_deact.head()
ebb_1_act_de = ebb_1_o_act.merge(ebb_1_deact, how='left', on='customer_id')
ebb_2_act_de = ebb_2_o_act.merge(ebb_2_deact, how='left', on='customer_id')
eval_act_de_ = eval_o_act.merge(eval_deact, how='left', on='customer_id')
ebb_1_act_de.head()

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,opt_out_loyalty_sms,opt_out_mobiles_ads,opt_out_phone,state,total_revenues_bucket,marketing_comms_1,marketing_comms_2,ebb_eligible,activation_counts,deactivation_counts
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,2.0,9.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,1.0,2.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,NaN,-1.0,NaN,TN,3,NaN,NaN,1.0,5.0,14.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,NaN,1.0,NaN,AL,1,NaN,NaN,1.0,1.0,9.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,NaN,1.0,NaN,MS,2,NaN,NaN,1.0,1.0,13.0


In [6]:
ebb_1_inter = pd.read_csv("interactions_ebb_set1.csv")
ebb_2_inter = pd.read_csv("interactions_ebb_set2.csv")
eval_inter = pd.read_csv("interactions_eval_set.csv")
ebb_1_inter.info()
#ebb_1_inter.head()
def count_times_inter(df):
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='interection_counts')
    df = counts.drop_duplicates(subset=['customer_id'])
    return df
ebb_1_inter = count_times_inter(ebb_1_inter)
ebb_2_inter = count_times_inter(ebb_2_inter)
eval_inter = count_times_inter(eval_inter)

#ebb_1_inter.head()
ebb_1_adi = ebb_1_act_de.merge(ebb_1_inter, how='left', on='customer_id')
ebb_2_adi = ebb_2_act_de.merge(ebb_2_inter, how='left', on='customer_id')
eval_adi = eval_act_de_.merge(eval_inter, how='left', on='customer_id')
ebb_1_adi.info()
ebb_1_adi.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206304 entries, 0 to 206303
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  206304 non-null  object
 1   date         206304 non-null  object
 2   issue        125245 non-null  object
 3   reason       199842 non-null  object
 4   disposition  205345 non-null  object
dtypes: object(5)
memory usage: 7.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   491

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,opt_out_mobiles_ads,opt_out_phone,state,total_revenues_bucket,marketing_comms_1,marketing_comms_2,ebb_eligible,activation_counts,deactivation_counts,interection_counts
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,0,NaN,NaN,1.0,2.0,9.0,3.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,0,NaN,NaN,1.0,1.0,2.0,5.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,-1.0,NaN,TN,3,NaN,NaN,1.0,5.0,14.0,6.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,1.0,NaN,AL,1,NaN,NaN,1.0,1.0,9.0,5.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,1.0,NaN,MS,2,NaN,NaN,1.0,1.0,13.0,36.0


In [7]:
ebb_1_ival = pd.read_csv("ivr_calls_ebb_set1.csv")
ebb_2_ival = pd.read_csv("ivr_calls_ebb_set2.csv")
eval_ival = pd.read_csv("ivr_calls_eval_set.csv")
ebb_1_ival.info()
#ebb_1_ival.head()

def count_times_ival(df):
    #how many calls
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='ival_counts')
    #iscompleted ->mean value
    df['iscompleted'] = df['iscompleted'].replace(0, -1)
    df['iscompleted'] = df['iscompleted'].fillna(0)
    df['iscompleted'] = df['iscompleted'].astype(int)
    df_kb = df['iscompleted'].groupby(df['customer_id'],sort = False).mean()
    df_kb = pd.DataFrame(df_kb).reset_index()  
    df_kb['iscompleted'][df_kb.iscompleted>0] = 1
    df_kb['iscompleted'][df_kb.iscompleted<0] = -1
    df = pd.merge(counts,df_kb)
    
    return df

ebb_1_ival = count_times_ival(ebb_1_ival)
ebb_2_ival = count_times_ival(ebb_2_ival)
eval_ival = count_times_ival(eval_ival)

#ebb_1_ival.head()
ebb_1_adii = ebb_1_adi.merge(ebb_1_ival, how='left', on='customer_id')
ebb_2_adii = ebb_2_adi.merge(ebb_2_ival, how='left', on='customer_id')
eval_adii = eval_adi.merge(eval_ival, how='left', on='customer_id')
ebb_1_adii.info()
ebb_1_adii.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942260 entries, 0 to 942259
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   customer_id      942260 non-null  object 
 1   call_start_time  942260 non-null  object 
 2   call_end_time    942260 non-null  object 
 3   iscompleted      776496 non-null  float64
dtypes: float64(1), object(3)
memory usage: 28.8+ MB


/tmp/ipykernel_52337/1682555811.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kb['iscompleted'][df_kb.iscompleted>0] = 1
/tmp/ipykernel_52337/1682555811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kb['iscompleted'][df_kb.iscompleted<0] = -1
/tmp/ipykernel_52337/1682555811.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kb['iscompleted'][df_kb.iscompleted>0] = 1
/tmp/ipykernel_52337/1682555811.py:17: Settin

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer           48764 non-null  object 
 8   operating_system       48764 non-null  object 
 9   language_preference    4741 non-null   object 
 10  opt_out_email          2856 non-null   float64
 11  opt_out_loyalty_email  79 non-null     float64
 12  opt_out_loyalty_sms    84 non-null     float64
 13  opt_out_mobiles_ads    24545 non-null  float64
 14  opt_out_phone          4311 non-null   float64
 15  st

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,state,total_revenues_bucket,marketing_comms_1,marketing_comms_2,ebb_eligible,activation_counts,deactivation_counts,interection_counts,ival_counts,iscompleted
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,0,NaN,NaN,1.0,2.0,9.0,3.0,37.0,-1.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,0,NaN,NaN,1.0,1.0,2.0,5.0,4.0,-1.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,TN,3,NaN,NaN,1.0,5.0,14.0,6.0,18.0,-1.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,AL,1,NaN,NaN,1.0,1.0,9.0,5.0,6.0,-1.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,MS,2,NaN,NaN,1.0,1.0,13.0,36.0,107.0,-1.0


In [8]:
ebb_1_net = pd.read_csv("network_ebb_set1.csv")
ebb_2_net = pd.read_csv("network_ebb_set2.csv")
eval_net = pd.read_csv("network_eval_set.csv")
ebb_1_net.info()
#ebb_1_net.head()

def convert_network(df):
    df_ka = df['total_kb'].groupby(df['customer_id'],sort = False).sum()
    df_ka = pd.DataFrame(df_ka).reset_index() 
    df_kb = df['voice_minutes'].groupby(df['customer_id'],sort = False).sum()
    df_kb = pd.DataFrame(df_kb).reset_index() 
    
    df_ka = df_ka.merge(df_kb, how='left', on='customer_id')
    
    df_kc = df['total_sms'].groupby(df['customer_id'],sort = False).sum()
    df_kc = pd.DataFrame(df_kc).reset_index() 
    
    df = df_ka.merge(df_kc, how='left', on='customer_id')
    return df 
ebb_1_net = convert_network(ebb_1_net)
ebb_2_net = convert_network(ebb_2_net)
eval_net = convert_network(eval_net)
#ebb_1_net.info()
#ebb_1_net.head()
ebb_1_adiin = ebb_1_adii.merge(ebb_1_net, how='left', on='customer_id')
ebb_2_adiin = ebb_2_adii.merge(ebb_2_net, how='left', on='customer_id')
eval_adiin = eval_adii.merge(eval_net, how='left', on='customer_id')
ebb_1_adiin.info()
ebb_1_adiin.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7016667 entries, 0 to 7016666
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_id    object 
 1   date           object 
 2   voice_minutes  int64  
 3   total_sms      int64  
 4   total_kb       float64
 5   hotspot_kb     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 321.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer      

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,marketing_comms_2,ebb_eligible,activation_counts,deactivation_counts,interection_counts,ival_counts,iscompleted,total_kb,voice_minutes,total_sms
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,1.0,2.0,9.0,3.0,37.0,-1.0,5.331525e+07,20762.0,4319.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,1.0,1.0,2.0,5.0,4.0,-1.0,3.109450e+07,4941.0,7352.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,NaN,1.0,5.0,14.0,6.0,18.0,-1.0,1.166038e+08,5785.0,9708.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,NaN,1.0,1.0,9.0,5.0,6.0,-1.0,1.696003e+08,21606.0,9251.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,NaN,1.0,1.0,13.0,36.0,107.0,-1.0,8.809456e+07,21790.0,8400.0


In [9]:
ebb_1_not = pd.read_csv("notifying_ebb_set1.csv")
ebb_2_not = pd.read_csv("notifying_ebb_set2.csv")
eval_not = pd.read_csv("notifying_eval_set.csv")
ebb_1_not.info()
#ebb_1_not.head()

def convert_notify_date(df):
    #how many notifiactions
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='notify_counts')
    #mean of date
    today_str = '2022-4-15'
    today = datetime.strptime(today_str, "%Y-%m-%d")
    df['notify_date']=pd.to_datetime(df['notify_date'],format='%Y %m %d')
    df['notify_date'] = today-df['notify_date']
    df['notify_date'] = df['notify_date'].astype('timedelta64[D]')
    df['notify_date'] = df['notify_date'].astype(int)
    df_kb = df['notify_date'].groupby(df['customer_id'],sort = False).mean() 
    df_kb = pd.DataFrame(df_kb).reset_index() 
    df_kb['notify_date'] = df_kb['notify_date'].astype(int)
    #df = pd.merge(counts,df_kb)
    df = counts.merge(df_kb, how='left', on='customer_id')

    return df
              
ebb_1_not = convert_notify_date(ebb_1_not)
ebb_2_not = convert_notify_date(ebb_2_not)
eval_not = convert_notify_date(eval_not)
#ebb_1_not.head()

ebb_1_adiinn = ebb_1_adiin.merge(ebb_1_not, how='left', on='customer_id')
ebb_2_adiinn = ebb_2_adiin.merge(ebb_2_not, how='left', on='customer_id')
eval_adiinn = eval_adiin.merge(eval_not, how='left', on='customer_id')
ebb_1_adiinn.info()
ebb_1_adiinn.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34945 entries, 0 to 34944
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  34945 non-null  object
 1   notify_date  34945 non-null  object
dtypes: object(2)
memory usage: 546.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer           48764 non-null  object 
 8   operating_system       48764 non-null  object 
 9   

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,activation_counts,deactivation_counts,interection_counts,ival_counts,iscompleted,total_kb,voice_minutes,total_sms,notify_counts,notify_date
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,2.0,9.0,3.0,37.0,-1.0,5.331525e+07,20762.0,4319.0,2.0,290.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,1.0,2.0,5.0,4.0,-1.0,3.109450e+07,4941.0,7352.0,NaN,NaN
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,5.0,14.0,6.0,18.0,-1.0,1.166038e+08,5785.0,9708.0,2.0,266.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,1.0,9.0,5.0,6.0,-1.0,1.696003e+08,21606.0,9251.0,5.0,239.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,1.0,13.0,36.0,107.0,-1.0,8.809456e+07,21790.0,8400.0,NaN,NaN


In [10]:
ebb_1_pho = pd.read_csv("phone_data_ebb_set1.csv")
ebb_2_pho = pd.read_csv("phone_data_ebb_set2.csv")
eval_pho = pd.read_csv("phone_data_eval_set.csv")
ebb_1_pho.info()
ebb_1_pho['battery_available'].value_counts()

def convert_pho(df):
    #battery_ava
    df['battery_available'][df.battery_available<0] = 0
    df_ka = df['battery_available'].groupby(df['customer_id'],sort = False).mean()
    df_ka = pd.DataFrame(df_ka).reset_index() 
    df_ka['battery_available'] = df_ka['battery_available'].astype(int)
    
    #data_roaming
    df['data_roaming'] = df['data_roaming'].replace([True,False], [1,-1])
    df['data_roaming'] = df['data_roaming'].fillna(0)
    df_kb = df['data_roaming'].groupby(df['customer_id'],sort = False).mean()
    df_kb = pd.DataFrame(df_kb).reset_index()  
    df_kb['data_roaming'][df_kb.data_roaming>0] = 1
    df_kb['data_roaming'][df_kb.data_roaming<0] = -1
    df_kb['data_roaming'] = df_kb['data_roaming'].astype(int)
    
    #df_1 = pd.merge(df_ka,df_kb)
    df_1 = df_ka.merge(df_kb, how='left', on='customer_id')
    #memory_ava
    df_kc = df['memory_available'].groupby(df['customer_id'],sort = False).mean()
    df_kc = pd.DataFrame(df_kc).reset_index() 
    df_kc['memory_available'] = df_kc['memory_available'].fillna(0)
    df_kc['memory_available'] = df_kc['memory_available'].astype(int)
    
    #df_2 = pd.merge(df_1,df_kc)
    df_2 = df_1.merge(df_kc, how='left', on='customer_id')
    #temperature
    df_kd = df['temperature'].groupby(df['customer_id'],sort = False).mean()
    df_kd = pd.DataFrame(df_kd).reset_index() 
    df_kd['temperature'] = df_kd['temperature'].fillna(0)
    df_kd['temperature'] = df_kd['temperature'].astype(int)
    
    df = df_2.merge(df_kd, how='left', on='customer_id')

    return df
              
ebb_1_pho = convert_pho(ebb_1_pho)
ebb_2_pho = convert_pho(ebb_2_pho)
eval_pho = convert_pho(eval_pho)

ebb_1_adiinnp = ebb_1_adiinn.merge(ebb_1_pho, how='left', on='customer_id')
ebb_2_adiinnp = ebb_2_adiinn.merge(ebb_2_pho, how='left', on='customer_id')
eval_adiinnp = eval_adiinn.merge(eval_pho, how='left', on='customer_id')
ebb_1_adiinnp.info()
ebb_1_adiinnp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1694838 entries, 0 to 1694837
Data columns (total 16 columns):
 #   Column                Dtype  
---  ------                -----  
 0   customer_id           object 
 1   battery_available     int64  
 2   battery_total         float64
 3   bluetooth_on          object 
 4   data_network_type     object 
 5   data_roaming          object 
 6   language              object 
 7   memory_available      int64  
 8   memory_total          int64  
 9   memory_treshold       int64  
 10  sd_storage_present    object 
 11  storage_available     int64  
 12  storage_total         float64
 13  temperature           float64
 14  time_since_last_boot  object 
 15  timestamp             object 
dtypes: float64(3), int64(5), object(8)
memory usage: 206.9+ MB


/tmp/ipykernel_52337/3517565021.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['battery_available'][df.battery_available<0] = 0
/tmp/ipykernel_52337/3517565021.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kb['data_roaming'][df_kb.data_roaming>0] = 1
/tmp/ipykernel_52337/3517565021.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kb['data_roaming'][df_kb.data_roaming<0] = -1
/tmp/ipykernel_52337/3517565021.py:9

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer           48764 non-null  object 
 8   operating_system       48764 non-null  object 
 9   language_preference    4741 non-null   object 
 10  opt_out_email          2856 non-null   float64
 11  opt_out_loyalty_email  79 non-null     float64
 12  opt_out_loyalty_sms    84 non-null     float64
 13  opt_out_mobiles_ads    24545 non-null  float64
 14  opt_out_phone          4311 non-null   float64
 15  st

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,iscompleted,total_kb,voice_minutes,total_sms,notify_counts,notify_date,battery_available,data_roaming,memory_available,temperature
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,-1.0,5.331525e+07,20762.0,4319.0,2.0,290.0,NaN,NaN,NaN,NaN
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,-1.0,3.109450e+07,4941.0,7352.0,NaN,NaN,NaN,NaN,NaN,NaN
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,-1.0,1.166038e+08,5785.0,9708.0,2.0,266.0,2978.0,1.0,604.0,83.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,-1.0,1.696003e+08,21606.0,9251.0,5.0,239.0,NaN,NaN,NaN,NaN
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,-1.0,8.809456e+07,21790.0,8400.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
ebb_1_rea = pd.read_csv("reactivations_ebb_set1.csv")
ebb_2_rea = pd.read_csv("reactivations_ebb_set2.csv")
eval_rea = pd.read_csv("reactivations_eval_set.csv")
ebb_1_rea.info()
#ebb_1_rea.head()

def count_times_rea(df):
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='reactivitaion_counts')
    df = counts.drop_duplicates(subset=['customer_id'])
    return df
ebb_1_rea = count_times_rea(ebb_1_rea)
ebb_2_rea = count_times_rea(ebb_2_rea)
eval_rea = count_times_rea(eval_rea)
#ebb_1_rea.head()

ebb_1_adiinnpr = ebb_1_adiinnp.merge(ebb_1_rea, how='left', on='customer_id')
ebb_2_adiinnpr = ebb_2_adiinnp.merge(ebb_2_rea, how='left', on='customer_id')
eval_adiinnpr = eval_adiinnp.merge(eval_rea, how='left', on='customer_id')
ebb_1_adiinnpr.info()
ebb_1_adiinnpr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363590 entries, 0 to 363589
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   customer_id           363590 non-null  object
 1   reactivation_date     363590 non-null  object
 2   reactivation_channel  363590 non-null  object
dtypes: object(3)
memory usage: 8.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer 

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,total_kb,voice_minutes,total_sms,notify_counts,notify_date,battery_available,data_roaming,memory_available,temperature,reactivitaion_counts
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,5.331525e+07,20762.0,4319.0,2.0,290.0,NaN,NaN,NaN,NaN,6.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,3.109450e+07,4941.0,7352.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,1.166038e+08,5785.0,9708.0,2.0,266.0,2978.0,1.0,604.0,83.0,10.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,1.696003e+08,21606.0,9251.0,5.0,239.0,NaN,NaN,NaN,NaN,9.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,8.809456e+07,21790.0,8400.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0


In [12]:
ebb_1_red = pd.read_csv("redemptions_ebb_set1.csv")
ebb_2_red = pd.read_csv("redemptions_ebb_set2.csv")
eval_red = pd.read_csv("redemptions_eval_set.csv")
ebb_1_red.info()
#ebb_1_red.head()

def convert_red_date(df):
    #how many redemption
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='redenption_counts')
    
    today_str = '2022-4-15'
    today = datetime.strptime(today_str, "%Y-%m-%d")
    df['date']= pd.to_datetime(df['date'],format='%Y %m %d')
    df['date'] = today-df['date']
    df['date'] = df['date'].astype('timedelta64[D]')
    df['date'] = df['date'].astype(int)
    df_kb= df['date'].groupby(df['customer_id'],sort = False).mean()
    df_kb = pd.DataFrame(df_kb).reset_index()
    df_kb['date'] = df_kb['date'].astype(int)
    #df = df_kb.join(df, how='right')
    df = counts.merge(df_kb, how='left', on='customer_id')
    return df
              
ebb_1_red = convert_red_date(ebb_1_red)
ebb_2_red = convert_red_date(ebb_2_red)
eval_red = convert_red_date(eval_red)
#ebb_1_red.head()

ebb_1_adiinnprr = ebb_1_adiinnpr.merge(ebb_1_red, how='left', on='customer_id')
ebb_2_adiinnprr = ebb_2_adiinnpr.merge(ebb_2_red, how='left', on='customer_id')
eval_adiinnprr = eval_adiinnpr.merge(eval_red, how='left', on='customer_id')
ebb_1_adiinnprr.info()
ebb_1_adiinnprr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107797 entries, 0 to 1107796
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   customer_id      1107797 non-null  object
 1   redemption_type  1107797 non-null  object
 2   date             1107797 non-null  object
 3   revenues         1107797 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 33.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,total_sms,notify_counts,notify_date,battery_available,data_roaming,memory_available,temperature,reactivitaion_counts,redenption_counts,date
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,4319.0,2.0,290.0,NaN,NaN,NaN,NaN,6.0,9,315
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,7352.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7,196
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,9708.0,2.0,266.0,2978.0,1.0,604.0,83.0,10.0,36,513
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,9251.0,5.0,239.0,NaN,NaN,NaN,NaN,9.0,25,292
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,8400.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,37,538


In [13]:
ebb_1_sup = pd.read_csv("support_ebb_set1.csv")
ebb_2_sup = pd.read_csv("support_ebb_set2.csv")
eval_sup = pd.read_csv("support_eval_set.csv")
#ebb_1_act.info()
#ebb_1_act.head()

def count_act_times(df):
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='support_counts')
    df = counts.drop_duplicates(subset=['customer_id'])
    return df

ebb_1_sup = count_act_times(ebb_1_sup)
ebb_2_sup = count_act_times(ebb_2_sup)
eval_sup = count_act_times(eval_sup)

#ebb_1_act.head()

ebb_1_o_sup = ebb_1_adiinnprr.merge(ebb_1_sup, how='left', on='customer_id')
ebb_2_o_sup = ebb_2_adiinnprr.merge(ebb_2_sup, how='left', on='customer_id')
eval_o_sup = eval_adiinnprr.merge(eval_sup, how='left', on='customer_id')
ebb_1_o_sup.head()

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,notify_counts,notify_date,battery_available,data_roaming,memory_available,temperature,reactivitaion_counts,redenption_counts,date,support_counts
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,2.0,290.0,NaN,NaN,NaN,NaN,6.0,9,315,2.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7,196,2.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,2.0,266.0,2978.0,1.0,604.0,83.0,10.0,36,513,5.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,5.0,239.0,NaN,NaN,NaN,NaN,9.0,25,292,1.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,37,538,4.0


In [14]:
ebb_1_thr = pd.read_csv("throttling_ebb_set1.csv")
ebb_2_thr = pd.read_csv("throttling_ebb_set2.csv")
eval_thr = pd.read_csv("throttling_eval_set.csv")
ebb_1_thr.info()
#ebb_1_thr.head()

def count_times_thr(df):
    #how many redemption
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='throttled_counts')
    
    today_str = '2022-4-15'
    today = datetime.strptime(today_str, "%Y-%m-%d")
    df['throttled_date']= pd.to_datetime(df['throttled_date'],format='%Y %m %d')
    df['throttled_date'] = today-df['throttled_date']
    df['throttled_date'] = df['throttled_date'].astype('timedelta64[D]')
    df['throttled_date'] = df['throttled_date'].astype(int)
    df_kb= df['throttled_date'].groupby(df['customer_id'],sort = False).mean()
    df_kb = pd.DataFrame(df_kb).reset_index()
    df_kb['throttled_date'] = df_kb['throttled_date'].astype(int)

    df = counts.merge(df_kb, how='left', on='customer_id')
    return df

ebb_1_thr = count_times_thr(ebb_1_thr)
ebb_2_thr = count_times_thr(ebb_2_thr)
eval_thr = count_times_thr(eval_thr)
#ebb_1_thr.head()

ebb_1_adiinnprrt = ebb_1_o_sup.merge(ebb_1_thr, how='left', on='customer_id')
ebb_2_adiinnprrt = ebb_2_o_sup.merge(ebb_2_thr, how='left', on='customer_id')
eval_adiinnprrt = eval_o_sup.merge(eval_thr, how='left', on='customer_id')
ebb_1_adiinnprrt.info()
ebb_1_adiinnprrt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54062 entries, 0 to 54061
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     54062 non-null  object
 1   throttled_date  54062 non-null  object
dtypes: object(2)
memory usage: 844.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer           48764 non-null  object 
 8   operating_system       48764 non-null  o

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,battery_available,data_roaming,memory_available,temperature,reactivitaion_counts,redenption_counts,date,support_counts,throttled_counts,throttled_date
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,6.0,9,315,2.0,3.0,300.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,NaN,NaN,NaN,1.0,7,196,2.0,NaN,NaN
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,2978.0,1.0,604.0,83.0,10.0,36,513,5.0,3.0,206.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,NaN,NaN,NaN,NaN,9.0,25,292,1.0,5.0,238.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,NaN,NaN,NaN,NaN,13.0,37,538,4.0,NaN,NaN


In [15]:
ebb_1_loy = pd.read_csv("loyalty_program_ebb_set1.csv")
ebb_2_loy = pd.read_csv("loyalty_program_ebb_set2.csv")
eval_loy = pd.read_csv("loyalty_program_eval_set.csv")
#ebb_1_act.info()
#ebb_1_act.head()

def count_act_times(df):
    counts = df['customer_id'].value_counts(sort=False).rename_axis('customer_id').reset_index(name='loy_counts')
    df = counts.drop_duplicates(subset=['customer_id'])
    return df

ebb_1_loy1 = count_act_times(ebb_1_loy)
ebb_2_loy1 = count_act_times(ebb_2_loy)
eval_loy1 = count_act_times(eval_loy)

ebb_1_loy = ebb_1_loy.merge(ebb_1_loy1, how='left', on='customer_id')
ebb_2_loy = ebb_2_loy.merge(ebb_2_loy1, how='left', on='customer_id')
eval_loy = eval_loy.merge(eval_loy1, how='left', on='customer_id')

for col in ebb_1_loy.columns:
    if col == 'lrp_enrolled' or col == 'date':
        ebb_1_loy = ebb_1_loy.drop(col, axis=1)
        ebb_2_loy = ebb_2_loy.drop(col, axis=1)
        eval_loy = eval_loy.drop(col, axis=1)
ebb_1_loy.head()

ebb_1_loyy = ebb_1_adiinnprrt.merge(ebb_1_loy, how='left', on='customer_id')
ebb_2_loyy = ebb_2_adiinnprrt.merge(ebb_2_loy, how='left', on='customer_id')
eval_loyy = eval_adiinnprrt.merge(eval_loy, how='left', on='customer_id')
ebb_1_loyy['total_quantity']=ebb_1_loyy['total_quantity'].apply(lambda x: 0 if pd.isnull(x) else x)
ebb_1_loyy['loy_counts']=ebb_1_loyy['loy_counts'].apply(lambda x: 0 if pd.isnull(x) else x)
ebb_1_loyy.info()
ebb_1_loyy.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49168 entries, 0 to 49167
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            49168 non-null  object 
 1   last_redemption_date   49168 non-null  object 
 2   first_activation_date  49168 non-null  object 
 3   total_redemptions      49168 non-null  int64  
 4   tenure                 49168 non-null  int64  
 5   number_upgrades        49168 non-null  int64  
 6   year                   49168 non-null  int64  
 7   manufacturer           48764 non-null  object 
 8   operating_system       48764 non-null  object 
 9   language_preference    4741 non-null   object 
 10  opt_out_email          2856 non-null   float64
 11  opt_out_loyalty_email  79 non-null     float64
 12  opt_out_loyalty_sms    84 non-null     float64
 13  opt_out_mobiles_ads    24545 non-null  float64
 14  opt_out_phone          4311 non-null   float64
 15  st

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,memory_available,temperature,reactivitaion_counts,redenption_counts,date,support_counts,throttled_counts,throttled_date,total_quantity,loy_counts
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,Android,NaN,...,NaN,NaN,6.0,9,315,2.0,3.0,300.0,0.0,0.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,Android,NaN,...,NaN,NaN,1.0,7,196,2.0,NaN,NaN,0.0,0.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,604.0,83.0,10.0,36,513,5.0,3.0,206.0,434.0,1.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,iOS,NaN,...,NaN,NaN,9.0,25,292,1.0,5.0,238.0,0.0,0.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,iOS,NaN,...,NaN,NaN,13.0,37,538,4.0,NaN,NaN,440.0,1.0


In [16]:
ebb_1 = ebb_1_loyy
ebb_2 = ebb_2_loyy
eval_set = eval_loyy

In [17]:
#########################################same, just for saving time##############################################
# Print files in current working directory 
#os.chdir("/data/team15/data")
#os.listdir(os.curdir)
#ebb_1 = pd.read_csv("ebb_set1.csv")
#ebb_2 = pd.read_csv("ebb_set2.csv")
#eval_set = pd.read_csv("eval_set.csv")


def convert_date(df):
    today_str = '2022-4-15'
    today = datetime.strptime(today_str, "%Y-%m-%d")
    df['last_redemption']=pd.to_datetime(df['last_redemption_date'],format='%Y %m %d')
    df['first_activation']=pd.to_datetime(df['first_activation_date'],format='%Y %m %d')
    df['redemption_duaration'] = df['last_redemption'] - df['first_activation']
    df['last_redemption'] = today-df['last_redemption']
    df['first_activation'] = today-df['first_activation']
    df['last_redemption'] = df['last_redemption'].astype('timedelta64[D]')
    df['first_activation'] = df['first_activation'].astype('timedelta64[D]')
    df['redemption_duaration'] = df['redemption_duaration'].astype('timedelta64[D]')
    return df

convert_date(ebb_1)
convert_date(ebb_2)
convert_date(eval_set)

def clean_operating_system(x):
    if pd.isna(x):
        return 'Not Known'
    t=x.lower()
    if re.search(r'\bandroid\b', t):
        return 'ANDROID'
    elif re.search(r'\bios\b', t):
        return 'IOS'
    elif t.find('not known')!=-1:
        return 'Not Known'
    return 'Other'

ebb_1['operating_system']=ebb_1['operating_system'].apply(clean_operating_system)
ebb_2['operating_system']=ebb_2['operating_system'].apply(clean_operating_system)
eval_set['operating_system']=eval_set['operating_system'].apply(clean_operating_system)
df=pd.concat([ebb_1,ebb_2])


In [18]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98389 entries, 0 to 49220
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            98389 non-null  object 
 1   last_redemption_date   98389 non-null  object 
 2   first_activation_date  98389 non-null  object 
 3   total_redemptions      98389 non-null  int64  
 4   tenure                 98389 non-null  int64  
 5   number_upgrades        98389 non-null  int64  
 6   year                   98389 non-null  int64  
 7   manufacturer           97571 non-null  object 
 8   operating_system       98389 non-null  object 
 9   language_preference    9394 non-null   object 
 10  opt_out_email          5619 non-null   float64
 11  opt_out_loyalty_email  159 non-null    float64
 12  opt_out_loyalty_sms    173 non-null    float64
 13  opt_out_mobiles_ads    49077 non-null  float64
 14  opt_out_phone          8573 non-null   float64
 15  st

,customer_id,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,language_preference,...,redenption_counts,date,support_counts,throttled_counts,throttled_date,total_quantity,loy_counts,last_redemption,first_activation,redemption_duaration
0,e3da0f9e3076cdf64decd7cb79dc174a51f874a3,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,ANDROID,NaN,...,9,315,2.0,3.0,300.0,0.0,0.0,201.0,435.0,234.0
1,169287d33c0df61d56b44a276b6357f8d378e35a,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,ANDROID,NaN,...,7,196,2.0,NaN,NaN,0.0,0.0,106.0,287.0,181.0
2,db830e033aeaf092be7a8ded20e9d1da2aca2ad8,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,NaN,...,36,513,5.0,3.0,206.0,434.0,1.0,18.0,1152.0,1134.0
3,ad8f9ba3561d82d87fb44abb36b1ee9f07787c9b,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,IOS,NaN,...,25,292,1.0,5.0,238.0,0.0,0.0,26.0,541.0,515.0
4,a1717a5eef8f8a6df9c2aed7de4512af3a365261,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,IOS,NaN,...,37,538,4.0,NaN,NaN,440.0,1.0,40.0,1032.0,992.0


In [19]:
temp_columns = []
for col in df.columns:
    counter = df[col].isnull().sum()
    ratio = counter / len(df[col])
    if ratio > 0.7:
         temp_columns.append(col)
df = df.drop(temp_columns, axis=1)
dfe = eval_set.drop(temp_columns, axis=1)

df['ebb_eligible']=df['ebb_eligible'].apply(lambda x: 0 if pd.isnull(x) else 1)
dfy=df[['ebb_eligible']]
dfx = df.drop('ebb_eligible', axis=1)
dfx = dfx.drop('customer_id', axis=1)
dfe = dfe.drop('customer_id', axis=1)
dfx.info()
dfx.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98389 entries, 0 to 49220
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   last_redemption_date   98389 non-null  object 
 1   first_activation_date  98389 non-null  object 
 2   total_redemptions      98389 non-null  int64  
 3   tenure                 98389 non-null  int64  
 4   number_upgrades        98389 non-null  int64  
 5   year                   98389 non-null  int64  
 6   manufacturer           97571 non-null  object 
 7   operating_system       98389 non-null  object 
 8   opt_out_mobiles_ads    49077 non-null  float64
 9   state                  74402 non-null  object 
 10  total_revenues_bucket  98389 non-null  int64  
 11  activation_counts      89904 non-null  float64
 12  deactivation_counts    92028 non-null  float64
 13  interection_counts     69065 non-null  float64
 14  ival_counts            83403 non-null  float64
 15  is

,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,opt_out_mobiles_ads,state,...,redenption_counts,date,support_counts,throttled_counts,throttled_date,total_quantity,loy_counts,last_redemption,first_activation,redemption_duaration
0,2021-09-26,2021-02-04,8,265,5,2017,TCL Communication Ltd,ANDROID,NaN,NaN,...,9,315,2.0,3.0,300.0,0.0,0.0,201.0,435.0,234.0
1,2021-12-30,2021-07-02,6,205,1,2019,TCL Communication Ltd,ANDROID,NaN,NaN,...,7,196,2.0,NaN,NaN,0.0,0.0,106.0,287.0,181.0
2,2022-03-28,2019-02-18,36,970,6,2020,ALCATEL,ANDROID,-1.0,TN,...,36,513,5.0,3.0,206.0,434.0,1.0,18.0,1152.0,1134.0
3,2022-03-20,2020-10-21,23,274,2,2017,Apple Inc,IOS,1.0,AL,...,25,292,1.0,5.0,238.0,0.0,0.0,26.0,541.0,515.0
4,2022-03-06,2019-06-18,34,1021,1,2017,Apple Inc,IOS,1.0,MS,...,37,538,4.0,NaN,NaN,440.0,1.0,40.0,1032.0,992.0


In [20]:
dfe.info()
dfe.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19849 entries, 0 to 19848
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   last_redemption_date   19849 non-null  object 
 1   first_activation_date  19849 non-null  object 
 2   total_redemptions      19849 non-null  int64  
 3   tenure                 19849 non-null  int64  
 4   number_upgrades        19849 non-null  int64  
 5   year                   19849 non-null  int64  
 6   manufacturer           19825 non-null  object 
 7   operating_system       19849 non-null  object 
 8   opt_out_mobiles_ads    9015 non-null   float64
 9   state                  19116 non-null  object 
 10  total_revenues_bucket  19849 non-null  int64  
 11  activation_counts      18903 non-null  float64
 12  deactivation_counts    18986 non-null  float64
 13  interection_counts     14162 non-null  float64
 14  ival_counts            16275 non-null  float64
 15  is

,last_redemption_date,first_activation_date,total_redemptions,tenure,number_upgrades,year,manufacturer,operating_system,opt_out_mobiles_ads,state,...,redenption_counts,date,support_counts,throttled_counts,throttled_date,total_quantity,loy_counts,last_redemption,first_activation,redemption_duaration
0,2022-03-26,2018-11-12,42,1074,3,2021,SAMSUNG INC,ANDROID,1.0,NM,...,42,635,2.0,NaN,NaN,6400.0,1.0,20.0,1250.0,1230.0
1,2022-03-26,2019-12-09,20,821,6,2021,SAMSUNG INC,ANDROID,1.0,FL,...,20,381,3.0,NaN,NaN,9888.0,1.0,20.0,858.0,838.0
2,2022-03-30,2017-12-31,49,1322,2,2021,SAMSUNG INC,ANDROID,-1.0,FL,...,49,734,1.0,NaN,NaN,650.0,1.0,16.0,1566.0,1550.0
3,2022-04-01,2018-03-21,49,1386,3,2021,SAMSUNG INC,ANDROID,NaN,OH,...,49,734,1.0,1.0,107.0,1895.0,1.0,14.0,1486.0,1472.0
4,2022-03-17,2017-05-20,15,1753,3,2021,SAMSUNG INC,ANDROID,1.0,FL,...,15,1110,1.0,NaN,NaN,1911.0,1.0,29.0,1791.0,1762.0


In [21]:
label = LabelEncoder()
def transform_to_int(df):
    for col in df:
        if col == 'operating_system' or col == 'state' or col == 'manufacturer' or col == 'last_redemption_date' or col == 'first_activation_date':
            df[col] = label.fit_transform(df[col])
transform_to_int(dfx)
transform_to_int(dfe)
dfx.head()
dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98389 entries, 0 to 49220
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   last_redemption_date   98389 non-null  int64  
 1   first_activation_date  98389 non-null  int64  
 2   total_redemptions      98389 non-null  int64  
 3   tenure                 98389 non-null  int64  
 4   number_upgrades        98389 non-null  int64  
 5   year                   98389 non-null  int64  
 6   manufacturer           98389 non-null  int64  
 7   operating_system       98389 non-null  int64  
 8   opt_out_mobiles_ads    49077 non-null  float64
 9   state                  98389 non-null  int64  
 10  total_revenues_bucket  98389 non-null  int64  
 11  activation_counts      89904 non-null  float64
 12  deactivation_counts    92028 non-null  float64
 13  interection_counts     69065 non-null  float64
 14  ival_counts            83403 non-null  float64
 15  is

In [22]:
def fill_nan(df):
    for col in df:
        df[col].fillna(df[col].median(), inplace=True)
fill_nan(dfx)

dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98389 entries, 0 to 49220
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   last_redemption_date   98389 non-null  int64  
 1   first_activation_date  98389 non-null  int64  
 2   total_redemptions      98389 non-null  int64  
 3   tenure                 98389 non-null  int64  
 4   number_upgrades        98389 non-null  int64  
 5   year                   98389 non-null  int64  
 6   manufacturer           98389 non-null  int64  
 7   operating_system       98389 non-null  int64  
 8   opt_out_mobiles_ads    98389 non-null  float64
 9   state                  98389 non-null  int64  
 10  total_revenues_bucket  98389 non-null  int64  
 11  activation_counts      98389 non-null  float64
 12  deactivation_counts    98389 non-null  float64
 13  interection_counts     98389 non-null  float64
 14  ival_counts            98389 non-null  float64
 15  is

In [23]:
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state = 1)

In [24]:
model = XGBClassifier()
model.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python

[17:18:41] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=64,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
y_pre = model.predict(X_test)
label = np.round(y_pre)
print("The f1 score is ", f1_score(y_test, label))

The f1 score is  0.8973488311349092


In [26]:
def getDateString():
    return pd.Timestamp.now().strftime('%Y-%m-%d')
'''
model: The model of sklearn
eval_df: The Dataframe of eval dataset
csv_path: The path of output csv, if ignored, the file of current date .csv will be outputed.
'''
def submit(model, eval_df, csv_path = None):
    if csv_path==None:
        csv_path = getDateString() + '.csv'
    df_eval = eval_df.drop(columns=['customer_id'])
    df_csv = pd.DataFrame()
    df_csv['customer_id'] = eval_df['customer_id']
    df_csv['egg_eligible'] = model.predict(df_eval.values)
    df_csv = df_csv.drop_duplicates(['customer_id'])
    df_csv.to_csv(csv_path, index=False)

'''
prediction_list: The prediction list like [0,1,0,0,1]
eval_df: The Dataframe of eval dataset
csv_path: The path of output csv, if ignored, the file of current date .csv will be outputed.
'''    
def submit_prediction(prediction_list, eval_df, csv_path = None):
    if csv_path==None:
        csv_path = getDateString() + '.csv'
    df_csv = pd.DataFrame()
    df_csv['customer_id'] = eval_df['customer_id']
    df_csv['egg_eligible'] = prediction_list
    df_csv = df_csv.drop_duplicates(['customer_id'])
    df_csv.to_csv(csv_path, index=False)

'''
prediction_list: The prediction list like [0,1,0,0,1]
customer_id: The customer id list
csv_path: The path of output csv, if ignored, the file of current date .csv will be outputed.
'''
def submit_prediction_id(prediction_list, customer_id, csv_path = None):
    if csv_path==None:
        csv_path = getDateString() + '.csv'
    df_csv['customer_id'] = customer_id
    df_csv['egg_eligible'] = prediction_list
    df_csv = df_csv.drop_duplicates(['customer_id'])
    df_csv.to_csv(csv_path, index=False)